In [ ]:
import io
import pandas as pd
import polars as pl
import msoffcrypto
from openpyxl import load_workbook

# --- CONFIGURACIÓ ---
ruta_excel = r'ruta/del/arxiu.xlsx'
tu_contraseña = '*****'

# --- Desxifrar arxiu ---
decrypted_workbook = io.BytesIO()
with open(ruta_excel, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=tu_contraseña)
    office_file.decrypt(decrypted_workbook)

# --- Obtindre noms dels fulls ---
decrypted_workbook.seek(0)
wb = load_workbook(decrypted_workbook, read_only=True)
nombres_hojas = wb.sheetnames
print("▶ Fulls trobats:")
for i, nombre in enumerate(nombres_hojas):
    print(f"{i + 1}: {nombre}")

# --- Llegir només el full número 5 ---
indice_hoja = 5  # Full 5 
nombre_hoja5 = nombres_hojas[indice_hoja]

decrypted_workbook.seek(0)
df_hoja5 = pd.read_excel(decrypted_workbook, sheet_name=nombre_hoja5, engine='openpyxl')
df = df_hoja5

# --- Mostrar primeres files del full 5 ---
print(f"\n Head del full 5: {nombre_hoja5}")
print(df_hoja5.head())

# PAS 1: Preparar el DataFrame
if isinstance(df, pl.DataFrame):
    df_pl = df
else:
    df_pl = pl.from_pandas(df)

print(f" Dataset original: {df_pl.shape}")

# PAS 2: Definir estratègia de fusió per a cada variable
def crear_columna_fusionada(df, columnas_candidatas, nombre_resultado):
    """Crea una columna fusionada prenent el primer valor no-null de les candidates"""
    columnas_existentes = [col for col in columnas_candidatas if col in df.columns]
    
    if not columnas_existentes:
        return df.with_columns(pl.lit(None).alias(nombre_resultado))
    
    # Utilitzar coalesce per a prendre el primer valor no-null
    expresion = pl.coalesce([pl.col(col) for col in columnas_existentes])
    return df.with_columns(expresion.alias(nombre_resultado))

# PAS 3: Fusionar totes les variables duplicades
print("\n FUSIONANT VARIABLES DUPLICADES:")

# Fusionar Albúmina
df_fusionado = crear_columna_fusionada(
    df_pl, 
    ['ALBÚMINA', 'resultadoALBÚMINA'], 
    'ALBUMINA_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['fechaResultado_ALBUMINA', 'fechaResultadoALBÚMINA'],
    'FECHA_ALBUMINA_FUSIONADA'
)

# Fusionar PCR
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['PCR', 'resultadoPCR'],
    'PCR_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['fechaResultadoPCR', 'fechaResultadoPCR1'],
    'FECHA_PCR_FUSIONADA'
)

# Fusionar Creatinina
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['resultadoCREATININA', 'resultadoCREATININA1'],
    'CREATININA_FUSIONADA'
)

# Per a creatinina només hi ha una data
df_fusionado = df_fusionado.with_columns(
    pl.col('fechaResultadoCREATININA').alias('FECHA_CREATININA_FUSIONADA')
)

# Fusionar dades demogràfiques duplicades
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descTipoPaciente', 'descTipoPaciente1'],
    'TIPO_PACIENTE_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descLinea', 'descLinea1'],
    'LINEA_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descEsquema', 'descEsquema1'],
    'ESQUEMA_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['codigoPostal', 'codigoPostal1'],
    'CODIGO_POSTAL_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descHospitalCrc', 'descHospitalCrc1'],
    'HOSPITAL_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descZona', 'descZona1'],
    'ZONA_FUSIONADA'
)

# PAS 4: Verificar quantas dades rescatem
print("DADES RESCATADES:")
print(f"Albúmina fusionada: {df_fusionado['ALBUMINA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"PCR fusionada: {df_fusionado['PCR_FUSIONADA'].drop_nulls().len():,} valors") 
print(f"Creatinina fusionada: {df_fusionado['CREATININA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"Data Albúmina fusionada: {df_fusionado['FECHA_ALBUMINA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"Data PCR fusionada: {df_fusionado['FECHA_PCR_FUSIONADA'].drop_nulls().len():,} valors")

# PAS 5: Crear el mapatge per a la transformació
variables_fusionadas = {
    'Rockwood': {'valor': 'Rockwood', 'fecha': 'F_Rockwood'},
    'ALBUMINA': {'valor': 'ALBUMINA_FUSIONADA', 'fecha': 'FECHA_ALBUMINA_FUSIONADA'},
    'PCR': {'valor': 'PCR_FUSIONADA', 'fecha': 'FECHA_PCR_FUSIONADA'},
    'CREATININA': {'valor': 'CREATININA_FUSIONADA', 'fecha': 'FECHA_CREATININA_FUSIONADA'},
    'EUROQOL': {'valor': 'descResultadoEscala1', 'fecha': 'fechaEUROQOL'}
}

# Columnes demogràfiques fusionades
cols_demograficas = ['SIPCOD', 'sexo', 'fecha Naci', 'fecha Exitus',
                     'TIPO_PACIENTE_FUSIONADO', 'LINEA_FUSIONADA', 'ESQUEMA_FUSIONADO',
                     'CODIGO_POSTAL_FUSIONADO', 'HOSPITAL_FUSIONADO', 'ZONA_FUSIONADA']

print(f"\n PREPARATS PER A TRANSFORMAR AMB TOTES LES VARIANTS FUSIONADES")
print(f"Variables a processar: {list(variables_fusionadas.keys())}")

# PAS 6: Transformació a format llarg amb dades fusionades
print("\n TRANSFORMANT A FORMAT LLARG:")

dfs_lista = []

# Processar variables simples (no EuroQol)
for variable, config in variables_fusionadas.items():
    if variable != 'EUROQOL':
        col_valor = config['valor']
        col_fecha = config['fecha']
        
        print(f"Processant {variable}...")
        
        # Filtrar només registres amb dades vàlides
        df_temp = df_fusionado.filter(
            (pl.col(col_valor).is_not_null()) & 
            (pl.col(col_fecha).is_not_null())
        ).select([
            pl.col('SIPCOD').cast(pl.Utf8),
            pl.col(col_fecha).cast(pl.Utf8).alias('Fecha'),
            pl.col(col_valor).cast(pl.Utf8).alias('Valor'),
            *[pl.col(col).cast(pl.Utf8) for col in cols_demograficas[1:]]
        ]).with_columns(
            pl.lit(variable).alias('Variable')
        )
        
        dfs_lista.append(df_temp)
        print(f"   {variable}: {len(df_temp):,} registres vàlids")

# Unir variables simples
if dfs_lista:
    df_simple_fusionado = pl.concat(dfs_lista)
    print(f"\n Variables simples unides: {len(df_simple_fusionado):,} files")
else:
    df_simple_fusionado = pl.DataFrame()

# PAS 7: Processar EuroQol
print("\nProcessant EuroQol...")
euroqol_config = variables_fusionadas['EUROQOL']

euroqol_df = df_fusionado.filter(
    (pl.col(euroqol_config['valor']).is_not_null()) & 
    (pl.col(euroqol_config['fecha']).is_not_null())
).select([
    pl.col('SIPCOD').cast(pl.Utf8),
    pl.col(euroqol_config['fecha']).cast(pl.Utf8).alias('Fecha'),
    pl.col(euroqol_config['valor']).cast(pl.Utf8).alias('Valor_Original'),
    *[pl.col(col).cast(pl.Utf8) for col in cols_demograficas[1:]]
])

print(f"Dataset EuroQol: {len(euroqol_df):,} files")

# Expandir EuroQol 
if len(euroqol_df) > 0:
    euroqol_expandido = euroqol_df.with_columns([
        pl.col('Valor_Original').str.strip_chars().str.split('/').list.first().alias('valor_limpio')
    ]).filter(
        pl.col('valor_limpio').is_not_null() & 
        (pl.col('valor_limpio').str.len_chars() == 5)
    ).with_columns([
        pl.col('valor_limpio').str.slice(0, 1).alias('EQ5D_Movilidad'),
        pl.col('valor_limpio').str.slice(1, 1).alias('EQ5D_CuidadoPersonal'), 
        pl.col('valor_limpio').str.slice(2, 1).alias('EQ5D_ActividadesHabituales'),
        pl.col('valor_limpio').str.slice(3, 1).alias('EQ5D_DolorMalestar'),
        pl.col('valor_limpio').str.slice(4, 1).alias('EQ5D_AnsiedadDepresion')
    ])
    
    dimensiones_eq5d = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales', 
                        'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']
    
    
    euroqol_largo = euroqol_expandido.unpivot(
        index=['SIPCOD', 'Fecha'] + cols_demograficas[1:],
        on=dimensiones_eq5d,
        variable_name='Variable',
        value_name='Valor'
    )
    print(f"   EuroQol expandit: {len(euroqol_largo):,} files")
else:
    euroqol_largo = pl.DataFrame()

# PAS 8: Unir tot
dataframes_finales = []
if len(df_simple_fusionado) > 0:
    dataframes_finales.append(df_simple_fusionado)
    print(f"\n Variables simples: {len(df_simple_fusionado):,} files")

if len(euroqol_largo) > 0:
    dataframes_finales.append(euroqol_largo)  
    print(f" EuroQol: {len(euroqol_largo):,} files")

if dataframes_finales:
    # Assegurar columnes consistents
    columnas_comunes = None
    for df in dataframes_finales:
        if columnas_comunes is None:
            columnas_comunes = df.columns
        else:
            columnas_comunes = [col for col in columnas_comunes if col in df.columns]
    
    print(f"Columnes comunes: {len(columnas_comunes)}")
    
    dataframes_reordenados = [df.select(columnas_comunes) for df in dataframes_finales]
    df_final_mejorado = pl.concat(dataframes_reordenados)
    
    print(f"\n RESULTAT MILLORAT: {len(df_final_mejorado):,} files totals")
    print(f" Pacients únics: {df_final_mejorado['SIPCOD'].n_unique():,}")
    
    # Ordenar
    df_final_mejorado = df_final_mejorado.sort(['SIPCOD', 'Fecha'])
    
    # Emplenar dades demogràfiques
    cols_rellenar = [col for col in cols_demograficas[1:] if col in df_final_mejorado.columns]
    if cols_rellenar:
        df_final_mejorado = df_final_mejorado.with_columns([
            pl.col(col).fill_null(strategy="forward").fill_null(strategy="backward").over('SIPCOD')
            for col in cols_rellenar
        ])
    
    # Eliminar duplicats
    df_final_mejorado = df_final_mejorado.unique()
    
    print(f" RESULTAT FINAL: {df_final_mejorado.shape}")
    
    # Mostrar distribució per variable
    print(f"\n DISTRIBUCIÓ FINAL PER VARIABLE:")
    conteo_final = df_final_mejorado.group_by('Variable').agg(pl.len().alias('count')).sort('count', descending=True)
    print(conteo_final)
    
    print(f"\n MILLORA ACONSEGUIDA:")
    print(f"Abans: ~33,805 files")
    print(f"Ara: {len(df_final_mejorado):,} files")
    if len(df_final_mejorado) > 0:
        mejora = len(df_final_mejorado) - 33805
        porcentaje = ((len(df_final_mejorado) / 33805) - 1) * 100
        print(f" Increment: +{mejora:,} files ({porcentaje:.1f}% més dades!)")
    
else:
    print(" No s'han pogut processar les dades")
    df_final_mejorado = pl.DataFrame()

print("\n Procés completat amb fusió intel·ligent de dades!")

# Assignar el resultat final al DataFrame principal
df = df_final_mejorado

# Classificació correcta de variables
variables_euroqol = df.filter(pl.col("Variable").str.starts_with("EQ5D"))["Variable"].unique()
variables_numericas = ["ALBUMINA", "PCR", "CREATININA", "Rockwood"]
variables_ordinales = variables_euroqol.to_list()

# Neteja amb càlculs temporals clau
df_clean = df.with_columns([
    pl.col("Fecha").str.to_datetime(),
    pl.col("fecha Naci").str.to_datetime(),
    pl.col("fecha Exitus").str.to_datetime(),
    
    # Valors segons tipus de variable
    pl.when(pl.col("Variable").is_in(variables_numericas))
      .then(pl.col("Valor").cast(pl.Float64, strict=False))
      .otherwise(None).alias("valor_numerico"),
    
    pl.when(pl.col("Variable").is_in(variables_ordinales))
      .then(pl.col("Valor").cast(pl.Int32, strict=False))
      .otherwise(None).alias("valor_ordinal")
]).with_columns([
    # Edat al moment de la mesura
    ((pl.col("Fecha") - pl.col("fecha Naci")).dt.total_days()).alias("edad_dias_medicion"),
    
    # Estat vital
    pl.col("fecha Exitus").is_not_null().alias("fallecido"),
    
    # Temps fins a la defunció (si mor) o fins a l'última mesura (si viu)
    pl.when(pl.col("fecha Exitus").is_not_null())
      .then((pl.col("fecha Exitus") - pl.col("Fecha")).dt.total_days())
      .otherwise(None).alias("dias_hasta_exitus"),
    
    # Temps de supervivència des de la mesura fins a l'esdeveniment o censura
    pl.when(pl.col("fecha Exitus").is_not_null())
      .then((pl.col("fecha Exitus") - pl.col("Fecha")).dt.total_days())
      .otherwise((pl.lit("2025-06-27").str.to_date() - pl.col("Fecha").dt.date()).dt.total_days())
      .alias("tiempo_supervivencia")
])

print("\n DATASET NET ")
print(f"Forma final: {df_clean.shape}")
print(f"Columnes disponibles: {df_clean.columns}")
print(f"Variables numèriques: {variables_numericas}")
print(f"Variables ordinals EuroQol: {variables_ordinales}")

In [ ]:
# Llistat de tipus de paciente a conservar
tipos_a_mantener = [
    "Crónico pluripatológico",
"Paliativo no oncológico",
"Paliativo oncológico"
]

# Aplicar el filtro en polars
df_filtrado_polars = df_clean.filter(
    pl.col("TIPO_PACIENTE_FUSIONADO").is_in(tipos_a_mantener)
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats
import polars as pl

# ANÀLISI EXPLORATORI
# ===================================================================

# Configuració de la data del període d'estudi
PERIODE_INICI = "2020"
PERIODE_FI = "2024"
PERIODE_COMPLET = f"{PERIODE_INICI}-{PERIODE_FI}"

# Filtrar per tipus de pacient específics
tipus_a_mantindre = [
    "Crónico pluripatológico",
    "Paliativo no oncológico", 
    "Paliativo oncológico"
]

# Verificar si df_clean existeix
try:
    print(f" Dataset trobat: {df_clean.shape}")
    
    # Aplicar el filtre en polars
    df_filtrat_polars = df_clean.filter(
        pl.col("TIPO_PACIENTE_FUSIONADO").is_in(tipus_a_mantindre)
    )
    df_clean = df_filtrat_polars
    print(f" Filtre aplicat correctament: {df_clean.shape}")
    
except NameError:
    print(" ERROR: La variable 'df_clean' no està definida.")
    print(" SOLUCIÓ: Executa primer el codi de processament de dades que inclou:")
    print("   1. Lectura del fitxer Excel")
    print("   2. Fusió de variables duplicades") 
    print("   3. Transformació a format llarg")
    print("   4. Netejat i preparació final")
    print("\n Després d'executar eixe codi, torna a executar aquest.")
    raise

# 1) Convertir df_clean filtrat (Polars) a pandas i crear df_pacients ÚNIC
df_pd = df_clean.to_pandas()
df_pd["edat_anys"] = df_pd["edad_dias_medicion"] / 365.25

# Crear un sol DataFrame 
df_pacients = (
    df_pd
    .groupby("SIPCOD")
    .agg({
        "edat_anys": "mean",
        "sexo": "first",
        "ZONA_FUSIONADA": "first",
        "fallecido": "first",
        "TIPO_PACIENTE_FUSIONADO": "first",
        "Fecha": ["min", "max"]  # Per calcular dies UHD
    })
    .reset_index()
)

# Aplanar columnes
df_pacients.columns = ['SIPCOD', 'edat_anys', 'sexo', 'ZONA_FUSIONADA', 'fallecido', 
                       'TIPO_PACIENTE_FUSIONADO', 'data_entrada', 'data_eixida']
# Calcular dies UHD
df_pacients['data_entrada'] = pd.to_datetime(df_pacients['data_entrada'])
df_pacients['data_eixida'] = pd.to_datetime(df_pacients['data_eixida'])
df_pacients['dies_UHD'] = (df_pacients['data_eixida'] - df_pacients['data_entrada']).dt.days + 1

# Netejar dades nul·les 
df_pacients = df_pacients.dropna(subset=['ZONA_FUSIONADA', 'sexo'])

# Nombre final de pacients
TOTAL_PACIENTS = len(df_pacients)
print(f" DATASET FINAL UNIFICAT: {TOTAL_PACIENTS:,} pacients")
print(f"Aquest número apareixerà en tots els gràfics per consistència.")

# Crear dos DataFrames: pacients difunts i supervivents
df_difunts = df_pd[df_pd['fallecido'] == True].copy()
df_supervivents = df_pd[df_pd['fallecido'] == False].copy()

# Vista ràpida amb informació del període
print(f" PERÍODE {PERIODE_COMPLET}")
print("="*60)
print(f"Total registres: {df_pd.shape[0]:,}")
print(f"Difunts: {df_difunts.shape[0]:,}")
print(f"Supervivents: {df_supervivents.shape[0]:,}")
print(f"Pacients únics: {df_pacients.shape[0]:,}")
print(f"Tipus de pacients inclosos: {', '.join(tipus_a_mantindre)}")

#ESQUEMA DE COLORS
# ===================================================================

sns.set_style("white")

estat_colors = {False: "#1f77b4",   # BLAU per SUPERVIVENTS 
                True: "#e377c2"}    # ROSA per DIFUNTS 

# TAULA DESCRIPTIVA
# ===================================================================

print(f"\n ESTADÍSTIQUES DESCRIPTIVES - PERÍODE {PERIODE_COMPLET}:")
print(f"Total de pacients únics: {len(df_pacients):,}")

# Estadístiques per estat vital
for status, etiqueta in [(False, "Supervivents"), (True, "Difunts")]:
    subset = df_pacients[df_pacients['fallecido'] == status]
    print(f"\n{etiqueta}: n={len(subset):,} ({len(subset)/len(df_pacients)*100:.1f}%)")
    print(f"  Edat mitjana: {subset['edat_anys'].mean():.1f} ± {subset['edat_anys'].std():.1f} anys")


# Test estadístic edat entre grups
if len(df_pacients[df_pacients['fallecido']==True]) >= 5 and len(df_pacients[df_pacients['fallecido']==False]) >= 5:
    edat_supervivents = df_pacients[df_pacients['fallecido']==False]['edat_anys'].dropna()
    edat_difunts = df_pacients[df_pacients['fallecido']==True]['edat_anys'].dropna()
    
    statistic, p_value = stats.mannwhitneyu(edat_difunts, edat_supervivents, alternative='two-sided')
    print(f"\n Test Mann-Whitney edat (supervivents vs difunts): p={p_value:.4f}")
    if p_value < 0.05:
        print("   → Diferència significativa en edat entre grups")
    else:
        print("   → No hi ha diferència significativa en edat")

# Distribució per tipus de pacient
print(f"\n DISTRIBUCIÓ PER TIPUS DE PACIENT:")
tipus_dist = df_pacients['TIPO_PACIENTE_FUSIONADO'].value_counts()
for tipus, count in tipus_dist.items():
    print(f"  {tipus}: {count:,} ({count/len(df_pacients)*100:.1f}%)")

# HISTOGRAMA D'EDAT GENERAL 
# ===================================================================

plt.figure(figsize=(12, 7))

# Histograma general de totes les edats
plt.hist(df_pacients['edat_anys'].dropna(), 
         bins=30, 
         alpha=0.8, 
         color='#4CAF50',  
         edgecolor='black',
         linewidth=0.5)

plt.xlabel("Edat (anys)", fontsize=12)
plt.ylabel("Nombre de pacients", fontsize=12)
plt.title(f"Distribució general d'edat\nPacients totals: {TOTAL_PACIENTS:,} | Període: {PERIODE_COMPLET}", 
          fontweight='bold', fontsize=14)
plt.grid(alpha=0.3)

# Afegir estadístiques al gràfic
edat_stats = df_pacients['edat_anys'].describe()
textstr = f'Mitjana: {edat_stats["mean"]:.1f} anys\n'
textstr += f'Mediana: {edat_stats["50%"]:.1f} anys\n'
textstr += f'Desviació estàndard: {edat_stats["std"]:.1f} anys\n'
textstr += f'Rang: {edat_stats["min"]:.0f} - {edat_stats["max"]:.0f} anys'
props = dict(boxstyle='round', facecolor='lightgreen', alpha=0.8)
plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()


# HISTOGRAMA D'EDAT PER ESTAT VITAL
# ===================================================================

plt.figure(figsize=(12, 7))

# Preparar dades per a l'histograma
edat_supervivents = df_pacients[df_pacients['fallecido']==False]['edat_anys']
edat_difunts = df_pacients[df_pacients['fallecido']==True]['edat_anys']

# Histograma amb la paleta de colors
plt.hist([edat_supervivents, edat_difunts], 
         bins=25, 
         alpha=0.7, 
         label=['Supervivents', 'Difunts'],
         color=[estat_colors[False], estat_colors[True]])

plt.xlabel("Edat (anys)", fontsize=12)
plt.ylabel("Nombre de pacients", fontsize=12)
plt.title(f"Distribució d'edat per estat vital\nPacients totals: {TOTAL_PACIENTS:,} | Període: {PERIODE_COMPLET}", 
          fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)

# Afegir estadístiques al gràfic
textstr = f'Supervivents: n={len(edat_supervivents):,} (mitjana: {edat_supervivents.mean():.1f} anys)\n'
textstr += f'Difunts: n={len(edat_difunts):,} (mitjana: {edat_difunts.mean():.1f} anys)'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()

# GRÀFIC DE BARRES DE SEXE 
# ===================================================================

fig, ax1 = plt.subplots(1, 1, figsize=(14, 7))

# Identificar els valors de sexe en les dades
valors_sexe_únics = df_pacients['sexo'].value_counts()

# Mappejar els valors segons el que trobem
if 1 in valors_sexe_únics.index and 2 in valors_sexe_únics.index:
    # Els valors són 1 i 2
    categories_sexe = [1, 2]
    etiquetes_sexe = ['Hòmens', 'Dones']  # Assumint 1=Hòmens, 2=Dones
elif '1' in valors_sexe_únics.index and '2' in valors_sexe_únics.index:
    # Els valors són '1' i '2' com a strings
    categories_sexe = ['1', '2']
    etiquetes_sexe = ['Hòmens', 'Dones']
elif 'H' in valors_sexe_únics.index and 'D' in valors_sexe_únics.index:
    # Els valors són H i D
    categories_sexe = ['H', 'D']
    etiquetes_sexe = ['Hòmens', 'Dones']
else:
    # Usar els valors que realment existeixen
    categories_sexe = list(valors_sexe_únics.index[:2])  # Agafar els dos primers
    etiquetes_sexe = [f'Grup {i+1}' for i in range(len(categories_sexe))]


colors_sexe = ['#87CEEB', '#FFB6C1']  # Blau clar i rosa clar

sexe_pacients = df_pacients['sexo'].value_counts()

sexe_counts = [sexe_pacients.get(cat, 0) for cat in categories_sexe]

bars1 = ax1.bar(etiquetes_sexe, sexe_counts, 
                color=colors_sexe, alpha=0.8, edgecolor='black', linewidth=1)
ax1.set_ylabel('Nombre de pacients', fontsize=12)
ax1.set_xlabel('Sexe', fontsize=12)
ax1.grid(axis='y', alpha=0.3)

# Afegir valors i percentatges
#total_supervivents = len(supervivents_data)
for i, (bar, val) in enumerate(zip(bars1, sexe_counts)):
    if val > 0:
        percentage = (val / len(df_pacients)) * 100
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(sexe_counts) * 0.02,
                f'{val:,}\n({percentage:.1f}%)', ha='center', va='bottom', 
                fontweight='bold', fontsize=11)

plt.suptitle(f'Distribució de sexe\nPacients totals: {TOTAL_PACIENTS:,} | Període: {PERIODE_COMPLET}', 
             fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()

# DIES EN LA UHD PER TIPUS DE PACIENT
# ===================================================================

# Crear DataFrame amb informació per pacient incloent dates
df_pacients_dates = (
    df_pd
    .groupby("SIPCOD")
    .agg({
        "edat_anys": "mean",
        "sexo": "first",
        "ZONA_FUSIONADA": "first",
        "fallecido": "first",
        "TIPO_PACIENTE_FUSIONADO": "first",
        "Fecha": ["min", "max"]  # Primera i última data
    })
    .reset_index()
)

# Aplanar columnes multi-nivell
df_pacients_dates.columns = ['SIPCOD', 'edat_anys', 'sexo', 'ZONA_FUSIONADA', 'fallecido', 
                             'TIPO_PACIENTE_FUSIONADO', 'data_entrada', 'data_eixida']
df_pacients_dates = df_pacients_dates.dropna(subset=["ZONA_FUSIONADA"])
# Convertir dates a datetime
df_pacients_dates['data_entrada'] = pd.to_datetime(df_pacients_dates['data_entrada'])
df_pacients_dates['data_eixida'] = pd.to_datetime(df_pacients_dates['data_eixida'])

# Calcular dies en la UHD
df_pacients_dates['dies_UHD'] = (df_pacients_dates['data_eixida'] - df_pacients_dates['data_entrada']).dt.days + 1

# Filtrar valors vàlids (dies >= 1)
df_pacients_dates = df_pacients_dates[df_pacients_dates['dies_UHD'] >= 1]

print(f"Pacients amb dades vàlides de dies UHD: {len(df_pacients_dates):,}")

# Crear el gràfic de dies en UHD per tipus
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# 1. BOXPLOT per tipus de pacient
tipus_pacients = df_pacients_dates['TIPO_PACIENTE_FUSIONADO'].unique()
dades_boxplot = []
etiquetes_boxplot = []

for tipus in tipus_pacients:
    dies_tipus = df_pacients_dates[df_pacients_dates['TIPO_PACIENTE_FUSIONADO'] == tipus]['dies_UHD']
    if len(dies_tipus) > 0:
        dades_boxplot.append(dies_tipus)
        etiquetes_boxplot.append(f"{tipus}\n(n={len(dies_tipus)})")

# Crear boxplot
box_plot = ax1.boxplot(dades_boxplot, labels=etiquetes_boxplot, patch_artist=True)

# Colors diferents per cada tipus
colors_tipus = ['#FF9999', '#66B3FF', '#99FF99']
for patch, color in zip(box_plot['boxes'], colors_tipus):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax1.set_ylabel('Dies en la UHD', fontsize=12, fontweight='bold')
ax1.set_xlabel('Tipus de pacient', fontsize=12, fontweight='bold')
ax1.set_title('Distribució de dies en la UHD per tipus de pacient\n(Boxplot)', 
              fontweight='bold', fontsize=14)
ax1.grid(axis='y', alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 2. HISTOGRAMA APILAT
ax2_colors = ['#FF9999', '#66B3FF', '#99FF99']
dies_per_tipus = []
etiquetes_hist = []

for i, tipus in enumerate(tipus_pacients):
    dies_tipus = df_pacients_dates[df_pacients_dates['TIPO_PACIENTE_FUSIONADO'] == tipus]['dies_UHD']
    if len(dies_tipus) > 0:
        dies_per_tipus.append(dies_tipus)
        etiquetes_hist.append(tipus)

# Crear histograma apilat
ax2.hist(dies_per_tipus, bins=30, alpha=0.7, label=etiquetes_hist, 
         color=ax2_colors[:len(dies_per_tipus)], edgecolor='black', linewidth=0.5)

ax2.set_xlabel('Dies en la UHD', fontsize=12, fontweight='bold')
ax2.set_ylabel('Nombre de pacients', fontsize=12, fontweight='bold')
ax2.set_title('Histograma de dies en la UHD per tipus de pacient', 
              fontweight='bold', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(axis='y', alpha=0.3)

plt.suptitle(f'Anàlisi de dies en la UHD\nPacients totals: {len(df_pacients_dates):,} | Període: {PERIODE_COMPLET}', 
             fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()


# TAULA ESTADÍSTIC DE DIES EN UHD
# ===================================================================

print(f"\n ESTADÍSTIQUES DE DIES EN LA UHD PER TIPUS DE PACIENT:")
print(f"{'='*80}")

for tipus in tipus_pacients:
    dies_tipus = df_pacients_dates[df_pacients_dates['TIPO_PACIENTE_FUSIONADO'] == tipus]['dies_UHD']
    
    if len(dies_tipus) > 0:
        print(f"\n {tipus}:")
        print(f"   Nombre de pacients: {len(dies_tipus):,}")
        print(f"   Mitjana: {dies_tipus.mean():.1f} dies")
        print(f"   Mediana: {dies_tipus.median():.1f} dies")
        print(f"   Desviació estàndard: {dies_tipus.std():.1f} dies")
        print(f"   Mínim: {dies_tipus.min()} dies")
        print(f"   Màxim: {dies_tipus.max()} dies")
        print(f"   Quartil 25%: {dies_tipus.quantile(0.25):.1f} dies")
        print(f"   Quartil 75%: {dies_tipus.quantile(0.75):.1f} dies")

# Comptar pacients per zona
df_zona_counts = (
    df_pacients
    .groupby("ZONA_FUSIONADA")
    .agg({"SIPCOD": "nunique"})
    .rename(columns={"SIPCOD": "num_pacients"})
    .reset_index()
)

# Filtrar zones amb més de 15 pacients 
llindar_pacients = 15
df_zona_grans = df_zona_counts[df_zona_counts['num_pacients'] > llindar_pacients]
df_zona_petites = df_zona_counts[df_zona_counts['num_pacients'] <= llindar_pacients]

# Crear categoria "Altres" si hi ha zones petites
if len(df_zona_petites) > 0:
    altres_count = df_zona_petites['num_pacients'].sum()
    df_altres = pd.DataFrame({"ZONA_FUSIONADA": ["Altres"], "num_pacients": [altres_count]})
    df_zona_final = pd.concat([df_zona_grans, df_altres], ignore_index=True)
else:
    df_zona_final = df_zona_grans.copy()

df_zona_final = df_zona_final.sort_values('num_pacients', ascending=True)

# Plot 
if len(df_zona_final) > 0:
    plt.figure(figsize=(max(12, len(df_zona_final) * 0.5), 7))
    
    cmap = LinearSegmentedColormap.from_list('verd_gris', ['#bfde8e', '#6E6D68'])
    norm = mcolors.Normalize(vmin=df_zona_final['num_pacients'].min(), 
                            vmax=df_zona_final['num_pacients'].max())
    colors = [cmap(norm(v)) for v in df_zona_final['num_pacients']]
    
    bars = plt.bar(range(len(df_zona_final)), df_zona_final['num_pacients'], color=colors)
    
    plt.xlabel('Zona de salut', fontsize=12)
    plt.ylabel('Nombre de pacients', fontsize=12)
    plt.title(f'Distribució de pacients per zona de salut (>{llindar_pacients} pacients)\n'
              f'Pacients totals: {TOTAL_PACIENTS:,} | Període: {PERIODE_COMPLET}', 
              fontweight='bold', fontsize=14)
    
    # Etiquetes
    plt.xticks(range(len(df_zona_final)), df_zona_final['ZONA_FUSIONADA'], 
               rotation=45, ha='right')
    
    # Afegir valors damunt de les barres
    for i, (bar, val) in enumerate(zip(bars, df_zona_final['num_pacients'])):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f"{val:,}", ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# TAXA DE MORTALITAT PER ZONES
# ===================================================================

# Calcular taxa de mortalitat per zona
mort_zona = []
for zona in df_zona_final['ZONA_FUSIONADA']:
    if zona == "Altres":
        # Per "Altres", calcular taxa ponderada
        zones_altres = df_zona_petites['ZONA_FUSIONADA'].tolist()
        pacients_altres = df_pacients[df_pacients['ZONA_FUSIONADA'].isin(zones_altres)]
        if len(pacients_altres) > 0:
            taxa = pacients_altres['fallecido'].mean() * 100
        else:
            taxa = 0
    else:
        # Per zones individuals
        pacients_zona = df_pacients[df_pacients['ZONA_FUSIONADA'] == zona]
        taxa = pacients_zona['fallecido'].mean() * 100 if len(pacients_zona) > 0 else 0
    
    mort_zona.append({'ZONA_FUSIONADA': zona, 'taxa_mortalitat': taxa})

mort_final = pd.DataFrame(mort_zona)

# Plot taxa de mortalitat amb colors blaus i rosa
if len(mort_final) > 0:
    plt.figure(figsize=(max(12, len(mort_final) * 0.5), 7))
    
    # Gradient de blau a rosa per taxa de mortalitat
    cmap2 = LinearSegmentedColormap.from_list('blau_rosa', ['#1f77b4', '#e377c2'])
    norm2 = mcolors.Normalize(vmin=mort_final['taxa_mortalitat'].min(), 
                             vmax=mort_final['taxa_mortalitat'].max())
    colors2 = [cmap2(norm2(v)) for v in mort_final['taxa_mortalitat']]
    
    bars = plt.bar(range(len(mort_final)), mort_final['taxa_mortalitat'], color=colors2)
    
    plt.xlabel('Zona de salut', fontsize=12)
    plt.ylabel('Taxa de mortalitat (%)', fontsize=12)
    plt.title(f'Taxa de mortalitat per zona de salut\n'
              f'Pacients totals: {TOTAL_PACIENTS:,} | Període: {PERIODE_COMPLET}', 
              fontweight='bold', fontsize=14)
    
    plt.xticks(range(len(mort_final)), mort_final['ZONA_FUSIONADA'], 
               rotation=45, ha='right')
    
    # Afegir valors damunt de les barres
    for i, (bar, val) in enumerate(zip(bars, mort_final['taxa_mortalitat'])):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f"{val:.1f}%", ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

#DISTRIBUCIÓ PER TIPUS DE PACIENT 
# ===================================================================

plt.figure(figsize=(12, 7))

# Comptar pacients per tipus
tipus_counts = df_pacients['TIPO_PACIENTE_FUSIONADO'].value_counts()

# Colors diferents per cada tipus
colors_tipus = ['#ff9999','#66b3ff', '#99ff99'][:len(tipus_counts)]

bars = plt.bar(range(len(tipus_counts)), tipus_counts.values, color=colors_tipus)

plt.xlabel('Tipus de pacient', fontsize=12)
plt.ylabel('Nombre de pacients', fontsize=12)
plt.title(f'Distribució per tipus de pacient\n'
          f'Pacients totals: {len(df_pacients):,} | Període: {PERIODE_COMPLET}', 
          fontweight='bold', fontsize=14)

plt.xticks(range(len(tipus_counts)), tipus_counts.index, rotation=45, ha='right')

# Afegir valors i percentatges damunt de les barres
for (bar, val) in zip(bars, tipus_counts.values):
    percentage = (val / len(df_pacients)) * 100
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
            f"{val:,}\n({percentage:.1f}%)", ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

mort_tipus = []
for tipus in tipus_pacients:
    # Per zones individuals
    pacients_mort = df_pacients[df_pacients['TIPO_PACIENTE_FUSIONADO'] == tipus]
    taxa = pacients_mort['fallecido'].mean() * 100 if len(pacients_mort) > 0 else 0
    
    mort_tipus.append({'TIPO_PACIENTE_FUSIONADO': tipus, 'taxa_mortalitat': taxa})

mort_final = pd.DataFrame(mort_tipus)

# Plot taxa de mortalitat
if len(mort_final) > 0:
    plt.figure(figsize=(max(12, len(mort_final) * 0.5), 7))

    colors2 = ['#ff9999','#66b3ff', '#99ff99'][:len(tipus_counts)]
    
    bars = plt.bar(range(len(mort_final)), mort_final['taxa_mortalitat'], color=colors2)
    
    plt.xlabel('Tipus de pacient', fontsize=12)
    plt.ylabel('Taxa de mortalitat (%)', fontsize=12)
    plt.title(f'Taxa de mortalitat per tipus de pacient\n'
              f'Pacients totals: {TOTAL_PACIENTS:,} | Període: {PERIODE_COMPLET}', 
              fontweight='bold', fontsize=14)
    
    plt.xticks(range(len(mort_final)), mort_final['TIPO_PACIENTE_FUSIONADO'], 
               rotation=45, ha='right')
    
    # Afegir valors damunt de les barres
    for i, (bar, val) in enumerate(zip(bars, mort_final['taxa_mortalitat'])):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f"{val:.1f}%", ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()


In [ ]:

# ANÀLISI DE FLUX DE PACIENTS I EPIDEMIOLOGIA
# ===================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuració del període d'estudi
PERIODE_INICI = "2020"
PERIODE_FI = "2024"
PERIODE_COMPLET = f"{PERIODE_INICI}-{PERIODE_FI}"

# Configuració de colors
estat_colors = {False: "#1f77b4", True: "#e377c2"}  # Blau=Supervivents, Rosa=Difunts
colors_flux = {
    'entren': '#2E8B57',      # Verd fosc per entrades
    'eixen': '#FF6B6B',       # Roig per eixides
    'prevalencia': '#4169E1',  # Blau per prevalència
    'incidencia': '#32CD32',   # Verd per incidència
    'mortalitat': '#DC143C'    # Roig fosc per mortalitat
}




# PREPARACIÓ DE DADES
# ===================================================================

# Assegurar que tenim les dades necessàries
if 'df_pd' not in locals():
    df_pd = df_clean.to_pandas()
    df_pd["edat_anys"] = df_pd["edad_dias_medicion"] / 365.25

# Convertir dates
df_pd['Fecha'] = pd.to_datetime(df_pd['Fecha'])
df_pd['any'] = df_pd['Fecha'].dt.year
df_pd['mes'] = df_pd['Fecha'].dt.month

# Crear DataFrame per pacient amb dates d'entrada i eixida
df_pacients_flux = df_pd.groupby('SIPCOD').agg({
    'Fecha': ['min', 'max'],
    'fallecido': 'first',
    'TIPO_PACIENTE_FUSIONADO': 'first',
    'sexo': 'first',
    'edat_anys': 'mean',
    "ZONA_FUSIONADA": 'first'
}).reset_index()

# Aplanar columnes
df_pacients_flux.columns = ['SIPCOD', 'data_entrada', 'data_eixida', 'fallecido', 
                           'tipus_pacient', 'sexo', 'edat_anys', 'ZONA_FUSIONADA']
df_pacients_flux = df_pacients_flux.dropna(subset=['ZONA_FUSIONADA','sexo', 'tipus_pacient'])
print(f" DATASET FLUX: {len(df_pacients_flux):,} pacients | Període: {PERIODE_COMPLET}")


# PACIENTS QUE ENTREN I PACIENTS QUE EIXEN
# ===================================================================

print("\n1. ANÀLISI DE FLUX: ENTRADES I EIXIDES")
print("-"*50)

# Calcular entrades i eixides per any
anys_analisi = list(range(2020, 2025))
flux_dades = []

for any in anys_analisi:
    # Pacients que entren (primera mesura en aquest any)
    entrades = len(df_pacients_flux[df_pacients_flux['data_entrada'].dt.year == any])
    
    # Pacients que ixen (última mesura en aquest any)
    eixides = len(df_pacients_flux[df_pacients_flux['data_eixida'].dt.year == any])
    
    flux_dades.append({
        'any': any,
        'entrades': entrades,
        'eixides': eixides,
        'flux_net': entrades - eixides
    })

df_flux = pd.DataFrame(flux_dades)

# Crear gràfica de flux
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Subplot 1: Entrades vs Eixides per any
x_pos = np.arange(len(anys_analisi))
width = 0.35

bars1 = ax1.bar(x_pos - width/2, df_flux['entrades'], width,
                label='Pacients que entren', color=colors_flux['entren'], alpha=0.8)
bars2 = ax1.bar(x_pos + width/2, df_flux['eixides'], width,
                label='Pacients que eixen', color=colors_flux['eixen'], alpha=0.8)

ax1.set_xlabel('Any', fontweight='bold', fontsize=12)
ax1.set_ylabel('Nombre de pacients', fontweight='bold', fontsize=12)
ax1.set_title(f'Flux de pacients: Entrades vs Eixides\n'
              f'Pacients totals: {len(df_pacients_flux):,} | Període: {PERIODE_COMPLET}',
              fontweight='bold', fontsize=14)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(anys_analisi)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, axis='y')

# Afegir valors a les barres
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax1.text(bar.get_x() + bar.get_width()/2., height + max(df_flux['entrades'].max(), df_flux['eixides'].max()) * 0.01,
                    f'{int(height)}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# Subplot 2: Flux net (entrades - eixides)
colors_net = ['green' if x >= 0 else 'red' for x in df_flux['flux_net']]
bars3 = ax2.bar(anys_analisi, df_flux['flux_net'], color=colors_net, alpha=0.7, edgecolor='black')

ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Any', fontweight='bold', fontsize=12)
ax2.set_ylabel('Flux net de pacients', fontweight='bold', fontsize=12)
ax2.set_title(f'Flux net de pacients per any\n'
              f'Total analitzat: {len(df_pacients_flux):,} | Període: {PERIODE_COMPLET}',
              fontweight='bold', fontsize=14)
ax2.grid(True, alpha=0.3, axis='y')

# Afegir valors a les barres
for bar, val in zip(bars3, df_flux['flux_net']):
    height = bar.get_height()
    va = 'bottom' if height >= 0 else 'top'
    offset = abs(height) * 0.05 if height >= 0 else -abs(height) * 0.05
    ax2.text(bar.get_x() + bar.get_width()/2., height + offset,
            f'{int(val):+d}', ha='center', va=va, fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

# Estadístiques de flux
print(f"Resum del flux de pacients:")
print(f"   Total entrades: {df_flux['entrades'].sum():,}")
print(f"   Total eixides: {df_flux['eixides'].sum():,}")
print(f"   Flux net total: {df_flux['flux_net'].sum():+,}")
print(f"   Any amb més entrades: {df_flux.loc[df_flux['entrades'].idxmax(), 'any']} ({df_flux['entrades'].max():,} pacients)")
print(f"   Any amb més eixides: {df_flux.loc[df_flux['eixides'].idxmax(), 'any']} ({df_flux['eixides'].max():,} pacients)")

# PREVALÈNCIA, INCIDÈNCIA I MORTALITAT
# ===================================================================

# Calcular indicadors epidemiològics per any
epi_dades = []

for any in anys_analisi:
    # PREVALÈNCIA: Pacients actius (amb mesures) durant l'any
    pacients_actius_any = df_pd[df_pd['any'] == any]['SIPCOD'].nunique()
    
    # INCIDÈNCIA: Nous pacients (primera mesura) durant l'any
    nous_pacients = len(df_pacients_flux[df_pacients_flux['data_entrada'].dt.year == any])
    
    # MORTALITAT: Pacients que van morir i tenien activitat durant l'any
    pacients_any_ids = df_pd[df_pd['any'] == any]['SIPCOD'].unique()
    morts_any = len(df_pacients_flux[
        (df_pacients_flux['SIPCOD'].isin(pacients_any_ids)) & 
        (df_pacients_flux['fallecido'] == True)
    ])
    
    # Taxa de mortalitat (% sobre pacients actius)
    taxa_mortalitat = (morts_any / pacients_actius_any * 100) if pacients_actius_any > 0 else 0
    
    epi_dades.append({
        'any': any,
        'prevalencia': pacients_actius_any,
        'incidencia': nous_pacients,
        'mortalitat_absoluta': morts_any,
        'taxa_mortalitat': taxa_mortalitat
    })

df_epi = pd.DataFrame(epi_dades)

# Crear gràfica epidemiològica
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Subplot 1: Prevalença, Incidència i Mortalitat absoluta
x_pos = np.arange(len(anys_analisi))
width = 0.25

bars1 = ax1.bar(x_pos - width, df_epi['prevalencia'], width,
                label='Prevalència (Totals)', color=colors_flux['prevalencia'], alpha=0.8)
bars2 = ax1.bar(x_pos, df_epi['incidencia'], width,
                label='Incidència (Nous)', color=colors_flux['incidencia'], alpha=0.8)
bars3 = ax1.bar(x_pos + width, df_epi['mortalitat_absoluta'], width,
                label='Mortalitat (Difunts)', color=colors_flux['mortalitat'], alpha=0.8)

ax1.set_xlabel('Any', fontweight='bold', fontsize=12)
ax1.set_ylabel('Nombre de pacients', fontweight='bold', fontsize=12)
ax1.set_title(f'Indicadors epidemiològics absoluts\n'
              f'Pacients totals: {len(df_pacients_flux):,} | Període: {PERIODE_COMPLET}',
              fontweight='bold', fontsize=14)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(anys_analisi)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')

# Afegir valors a les barres
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            max_height = max(df_epi['prevalencia'].max(), df_epi['incidencia'].max(), df_epi['mortalitat_absoluta'].max())
            ax1.text(bar.get_x() + bar.get_width()/2., height + max_height * 0.01,
                    f'{int(height)}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# Subplot 2: Taxa de mortalitat i tendències
ax2_color = colors_flux['mortalitat']
line1 = ax2.plot(anys_analisi, df_epi['taxa_mortalitat'], 
                 color=ax2_color, marker='o', linewidth=3, markersize=8, 
                 label='Taxa de mortalitat (%)')

ax2.set_xlabel('Any', fontweight='bold', fontsize=12)
ax2.set_ylabel('Taxa de mortalitat (%)', fontweight='bold', fontsize=12, color=ax2_color)
ax2.set_title(f'Evolució de la taxa de mortalitat\n'
              f'Mitjana del període: {df_epi["taxa_mortalitat"].mean():.1f}% | Període: {PERIODE_COMPLET}',
              fontweight='bold', fontsize=14)
ax2.tick_params(axis='y', labelcolor=ax2_color)
ax2.grid(True, alpha=0.3)

# Afegir valors als punts
for x, y in zip(anys_analisi, df_epi['taxa_mortalitat']):
    ax2.text(x, y + max(df_epi['taxa_mortalitat']) * 0.05, f'{y:.1f}%', 
            ha='center', va='bottom', fontweight='bold', fontsize=10)

# Eix secundari per incidència
ax2_twin = ax2.twinx()
ax2_twin_color = colors_flux['incidencia']
line2 = ax2_twin.plot(anys_analisi, df_epi['incidencia'], 
                      color=ax2_twin_color, marker='s', linewidth=2, markersize=6, 
                      linestyle='--', label='Incidència (nous pacients)')

ax2_twin.set_ylabel('Incidència (nous pacients)', fontweight='bold', fontsize=12, color=ax2_twin_color)
ax2_twin.tick_params(axis='y', labelcolor=ax2_twin_color)

# Llegenda combinada
lines1, labels1 = ax2.get_legend_handles_labels()
lines2, labels2 = ax2_twin.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=10)

plt.tight_layout()
plt.show()

# Estadístiques epidemiològiques
print(f"Resum dels indicadors epidemiològics:")
print(f"   Prevalença mitjana: {df_epi['prevalencia'].mean():.0f} pacients/any")
print(f"   Incidència mitjana: {df_epi['incidencia'].mean():.0f} nous pacients/any")
print(f"   Taxa de mortalitat mitjana: {df_epi['taxa_mortalitat'].mean():.1f}%")
print(f"   Any amb major prevalença: {df_epi.loc[df_epi['prevalencia'].idxmax(), 'any']} ({df_epi['prevalencia'].max():,} pacients)")
print(f"   Any amb major incidència: {df_epi.loc[df_epi['incidencia'].idxmax(), 'any']} ({df_epi['incidencia'].max():,} nous pacients)")
print(f"   Any amb major mortalitat: {df_epi.loc[df_epi['taxa_mortalitat'].idxmax(), 'any']} ({df_epi['taxa_mortalitat'].max():.1f}%)")
